In [30]:
# Imports
import subprocess
import os

In [2]:
# Create pdbqt files of receptor and ligand using obabel
!obabel 1A4G-receptor.pdb -xr -O 1A4G-receptor.pdbqt
!obabel 1A4G-ZMR.pdb -O 1A4G-ZMR.pdbqt

1 molecule converted
1 molecule converted


In [3]:
# The ligand and receptor preparation for 1A4G was done following this guide: https://www.cheminformania.com/ligand-docking-with-smina/
# Receptor: 1A4G-receptor.pdbqt
# Ligand: 1A4G-ZMR.pdbqt
# Used parameters for box size and box center found here: https://autodock-vina.readthedocs.io/en/latest/docking_basic.html
cmd = [
    'vina',  
    '--receptor', '1A4G-receptor.pdbqt',
    '--ligand', '1A4G-ZMR.pdbqt',
    '--center_x', '15.190',  # specify box center coordinates (float)
    '--center_y', '53.903',
    '--center_z', '16.917',
    '--size_x', '20.0',
    '--size_y', '20.0',
    '--size_z', '20.0',
    '--exhaustiveness', '16',
    '--out', 'vina_docked.pdbqt'
]

subprocess.run(cmd)
print("Docking done via command line")

AutoDock Vina v1.2.7
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [43]:
# Function to extract the energetically most favorable candidate from the vina_docked file
def extract_top_pose(vina_pdbqt_path, output_path):
    with open(vina_pdbqt_path) as f_in, open(output_path, 'w') as f_out:
        write = False
        for line in f_in:
            if line.startswith('REMARK VINA RESULT:'):
                if write:  # we've already found first pose, stop here
                    break
                else:
                    write = True  # start writing pose from here
            if write:
                f_out.write(line)


In [45]:
# Extract top candidate
extract_top_pose('vina_docked.pdbqt', 'top_pose.pdbqt')
!obabel top_pose.pdbqt -O top_pose.pdb


1 molecule converted


In [46]:
# Compute RMSD of the energetically most favourable candidate
!python3 compute_rmsd.py -i 1A4G-ZMR.pdb -i2 top_pose.pdb

RMSD: 0.43 Å


In [17]:
# To check if there is a candidate with a better RMSD score, we split the mulit-pose file generated by AutoDock Vine
def split_pdbqt(filename, molecule_pdb):
    with open(filename, 'r') as f:
        lines = f.readlines()

    pose_num = 0
    pose_lines = []
    for line in lines:
        if line.startswith("MODEL"):
            pose_lines = [line]
        elif line.startswith("ENDMDL"):
            pose_lines.append(line)
            pose_num += 1
            out_name = f"{molecule_pdb}_Candidates/pose_{pose_num}.pdbqt"
            with open(out_name, "w") as out_file:
                out_file.writelines(pose_lines)
            print(f"Saved {out_name}")
        else:
            pose_lines.append(line)


split_pdbqt("vina_docked.pdbqt", "1A4G")


Saved 1A4G_Candidates/pose_1.pdbqt
Saved 1A4G_Candidates/pose_2.pdbqt
Saved 1A4G_Candidates/pose_3.pdbqt
Saved 1A4G_Candidates/pose_4.pdbqt
Saved 1A4G_Candidates/pose_5.pdbqt
Saved 1A4G_Candidates/pose_6.pdbqt
Saved 1A4G_Candidates/pose_7.pdbqt
Saved 1A4G_Candidates/pose_8.pdbqt
Saved 1A4G_Candidates/pose_9.pdbqt


In [42]:
!python3 compute_multiple_rmsd.py -i 1A4G-ZMR.pdb -i2 1A4G_Candidates/ # in fact there are poses with a lower RMSD than the energetically most favorable pose

Computing RMSD for 9 poses:
Pose 1: RMSD = 0.43 Å
Pose 2: RMSD = 0.51 Å
Pose 3: RMSD = 0.56 Å
Pose 4: RMSD = 0.25 Å
Pose 5: RMSD = 0.66 Å
Pose 6: RMSD = 0.23 Å
Pose 7: RMSD = 0.16 Å
Pose 8: RMSD = 0.53 Å
Pose 9: RMSD = 0.25 Å

Best RMSD: 0.16 Å at pose 7
